<a href="https://colab.research.google.com/github/lucasmedss/peca/blob/main/02_previsao_consumo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução

Este notebook gera previsão de consumo de energia em intervalos de **15 minutos** para um mês-alvo.

A ideia é simples: usar o histórico da própria edificação para criar um perfil de consumo por **dia da semana + horário**.

No final, o notebook compara **real x previsto** e mostra métricas de erro para apoiar análise do resultado.


# 1. Preparação

Nesta etapa carregamos as bibliotecas necessárias para:
- leitura e tratamento dos dados,
- geração dos gráficos,
- salvamento do resultado no Google Drive (quando usado no Colab).


In [1]:
# Bibliotecas principais

import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go


# 2. Parâmetros

Defina aqui o cenário da previsão:
- `TARGET_YEAR` e `TARGET_MONTH`: período a prever,
- `BUILDING_FILTER`: edificação alvo,
- `INPUT_CSV`: origem dos dados históricos,
- `OUTPUT_CSV`: caminho do arquivo final com a previsão.

Se quiser testar outro caso, normalmente basta alterar esta seção.


In [ ]:
# Parâmetros de previsão

TARGET_YEAR = 2025
TARGET_MONTH = 10
BUILDING_FILTER = "ESCOLA_MUNICIPAL"

INPUT_CSV = "https://raw.githubusercontent.com/lucasmedss/peca/main/cg_public_energy_consumption.csv"
df_raw = pd.read_csv(INPUT_CSV, parse_dates=["timestamp"])

OUTPUT_CSV = "/content/forecast/forecast_next_month_15min.csv"
TARGET_COL = "consumption_kwh"
BUILDING_COL = "building"


In [ ]:
def load_history(
    dataframe: pd.DataFrame,
    target_col: str,
    building_col: str = "building",
    building_filter: str | None = None
) -> pd.DataFrame:
    if target_col not in dataframe.columns:
        raise ValueError(f"CSV sem a coluna alvo: {target_col}")

    df = dataframe.copy()

    # Compatibilidade entre formato novo (building) e antigo (building_id)
    if building_col not in df.columns and "building_id" in df.columns:
        df[building_col] = df["building_id"]

    if building_col not in df.columns:
        raise ValueError(f"CSV sem a coluna de prédio esperada: {building_col}")

    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
    df[target_col] = pd.to_numeric(df[target_col], errors="coerce")
    df = df.dropna(subset=["timestamp", target_col])

    if building_filter is not None:
        filtro = str(building_filter).strip()
        df = df[df[building_col].astype(str).str.strip() == filtro]

    if df.empty:
        opcoes = sorted(dataframe[building_col].dropna().astype(str).str.strip().unique().tolist()) if building_col in dataframe.columns else []
        raise ValueError(
            "Sem dados após aplicar filtros e limpeza. "
            f"BUILDING_FILTER atual: {building_filter!r}. "
            f"Opções disponíveis: {opcoes}"
        )

    history = (
        df.sort_values("timestamp")
        .set_index("timestamp")[target_col]
        .resample("15min")
        .sum(min_count=1)
        .to_frame(name=target_col)
        .dropna()
    )

    return history


history = load_history(df_raw, TARGET_COL, BUILDING_COL, BUILDING_FILTER)

print("Dimensão do dataset bruto:", df_raw.shape)
print("Registros no histórico reamostrado:", len(history))
print("Período do histórico:", history.index.min(), "até", history.index.max())
history.head()


Dimensão do dataset bruto: (314837, 3)
Registros no histórico reamostrado: 52704
Período do histórico: 2024-07-01 00:00:00 até 2025-12-31 23:45:00


,consumption_kwh
timestamp,
2024-07-01 00:00:00,2.640983
2024-07-01 00:15:00,2.616166
2024-07-01 00:30:00,2.637341
2024-07-01 00:45:00,2.442556
2024-07-01 01:00:00,2.601213


# 3. Carga e preparação do histórico

A função desta etapa:
- valida as colunas esperadas,
- aplica filtro de edificação,
- converte tipos (`timestamp` e consumo),
- reamostra os dados para frequência de 15 minutos.

Isso garante uma base consistente para o cálculo da previsão.


In [32]:
def build_profile(history: pd.DataFrame, target_col: str):
    hist = history.copy()
    hist["weekday"] = hist.index.dayofweek
    hist["slot_15m"] = hist.index.hour * 4 + (hist.index.minute // 15)

    profile_weekday_slot = hist.groupby(["weekday", "slot_15m"])[target_col].mean()
    profile_slot = hist.groupby("slot_15m")[target_col].mean()
    global_mean = float(hist[target_col].mean())
    return profile_weekday_slot, profile_slot, global_mean


# 4. Perfil de consumo

Criamos três referências de consumo a partir do histórico:
- média por **dia da semana + slot de 15 minutos** (referência principal),
- média por **slot de 15 minutos** (fallback),
- **média global** (fallback final).

Esse encadeamento evita falhas quando faltar histórico em algum horário específico.


In [33]:
def month_range(year: int, month: int) -> pd.DatetimeIndex:
    start = pd.Timestamp(year=year, month=month, day=1)
    end = (start + pd.offsets.MonthEnd(1)).normalize() + pd.Timedelta(hours=23, minutes=45)
    return pd.date_range(start=start, end=end, freq="15min")

future_idx = month_range(TARGET_YEAR, TARGET_MONTH)
print("Período previsto:", future_idx.min(), "até", future_idx.max())


Período previsto: 2025-10-01 00:00:00 até 2025-10-31 23:45:00


# 5. Janela da previsão

Aqui montamos todos os timestamps do mês alvo em passos de 15 minutos.

Essa janela será a base sobre a qual o modelo aplica o perfil histórico e gera os valores previstos.


In [34]:
def forecast_month(
    future_idx: pd.DatetimeIndex,
    profile_weekday_slot: pd.Series,
    profile_slot: pd.Series,
    global_mean: float
) -> pd.DataFrame:
    future = pd.DataFrame(index=future_idx)
    future["weekday"] = future.index.dayofweek
    future["slot_15m"] = future.index.hour * 4 + (future.index.minute // 15)

    def predict_row(row):
        key = (int(row["weekday"]), int(row["slot_15m"]))
        if key in profile_weekday_slot.index:
            return float(profile_weekday_slot.loc[key])
        slot = int(row["slot_15m"])
        if slot in profile_slot.index:
            return float(profile_slot.loc[slot])
        return global_mean

    future["consumption_kwh_pred"] = future.apply(predict_row, axis=1)
    return future

profile_weekday_slot, profile_slot, global_mean = build_profile(history, TARGET_COL)
forecast = forecast_month(future_idx, profile_weekday_slot, profile_slot, global_mean)
forecast.head()


,weekday,slot_15m,consumption_kwh_pred
2025-10-01 00:00:00,2,0,2.617044
2025-10-01 00:15:00,2,1,2.552773
2025-10-01 00:30:00,2,2,2.493784
2025-10-01 00:45:00,2,3,2.469506
2025-10-01 01:00:00,2,4,2.451890


# 6. Geração da previsão mensal

Para cada ponto da janela futura, a previsão segue esta ordem:
1. usa média de mesmo dia da semana + horário,
2. se não existir, usa média do mesmo horário,
3. se ainda não existir, usa média global.

Na célula seguinte, o resultado é salvo em CSV.


In [ ]:
os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)
forecast.to_csv(OUTPUT_CSV, index_label="timestamp")
print(f"Forecast salvo em: {OUTPUT_CSV}")


# 7. Avaliação mensal e visão anual

Depois de gerar a previsão, o notebook compara valores **reais** e **previstos** por mês no ano de análise.

Além do gráfico, também é criada uma tabela com erro absoluto e erro percentual para facilitar leitura e discussão dos resultados.


In [36]:
# Visão anual (Real x Previsão)
ano_analise = TARGET_YEAR
future_idx_annual = pd.date_range(
    start=pd.Timestamp(year=ano_analise, month=1, day=1),
    end=pd.Timestamp(year=ano_analise, month=12, day=31, hour=23, minute=45),
    freq="15min"
)

forecast_annual = forecast_month(future_idx_annual, profile_weekday_slot, profile_slot, global_mean)
forecast_monthly = forecast_annual["consumption_kwh_pred"].resample("ME").sum()

actual_annual = history.loc[future_idx_annual.min():future_idx_annual.max(), [TARGET_COL]]
actual_monthly = actual_annual[TARGET_COL].resample("ME").sum() if not actual_annual.empty else pd.Series(dtype=float)

df_plot = pd.DataFrame(index=forecast_monthly.index)
df_plot["Previsto"] = forecast_monthly.values
df_plot["Real"] = actual_monthly.reindex(df_plot.index)

MESES_PT = {1: "Jan", 2: "Fev", 3: "Mar", 4: "Abr", 5: "Mai", 6: "Jun", 7: "Jul", 8: "Ago", 9: "Set", 10: "Out", 11: "Nov", 12: "Dez"}
df_plot["Mes_Nome"] = df_plot.index.month.map(MESES_PT)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_plot["Mes_Nome"],
    y=df_plot["Real"],
    mode="lines+markers",
    name="Consumo Real",
    line=dict(color="#3b82f6", width=3),
    fill="tozeroy",
    fillcolor="rgba(59, 130, 246, 0.1)"
))
fig.add_trace(go.Scatter(
    x=df_plot["Mes_Nome"],
    y=df_plot["Previsto"],
    mode="lines+markers",
    name="Previsão",
    line=dict(color="#ef4444", width=3, dash="dot"),
    marker=dict(symbol="x", size=8)
))

fig.update_layout(
    title=f"<b>Análise Anual ({ano_analise}): Real vs. Previsão</b><br><sup>Edificação: {BUILDING_FILTER}</sup>",
    xaxis_title="Mês do Ano",
    yaxis_title="Consumo Total Mensal (kWh)",
    plot_bgcolor="white",
    hovermode="x unified",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)
fig.update_yaxes(gridcolor="whitesmoke", tickformat=",")
fig.update_xaxes(showgrid=False)
fig.show()


In [37]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

df_metrics_base = pd.DataFrame()
df_metrics_base["Mês"] = df_plot["Mes_Nome"]
df_metrics_base["Consumo Real (kWh)"] = df_plot["Real"]
df_metrics_base["Consumo Previsto (kWh)"] = df_plot["Previsto"]


df_metrics_base["Erro (kWh)"] = df_metrics_base["Consumo Previsto (kWh)"] - df_metrics_base["Consumo Real (kWh)"]
df_metrics_base["Erro Percentual (%)"] = np.where(
    (df_metrics_base["Consumo Real (kWh)"].notna()) & (df_metrics_base["Consumo Real (kWh)"] != 0),
    (df_metrics_base["Erro (kWh)"] / df_metrics_base["Consumo Real (kWh)"]) * 100,
    np.nan
)

df_metrics_base = df_metrics_base.round({
    "Consumo Real (kWh)": 2,
    "Consumo Previsto (kWh)": 2,
    "Erro (kWh)": 2,
    "Erro Percentual (%)": 2
})

def format_br(val):
    if pd.isna(val):
        return "-"
    return f"{val:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

def format_perc(val):
    if pd.isna(val):
        return "-"
    return f"{val:.2f}%".replace(".", ",")


display(df_metrics_base)

,Mês,Consumo Real (kWh),Consumo Previsto (kWh),Erro (kWh),Erro Percentual (%)
2025-01-31,Jan,9474.02,8174.98,-1299.04,-13.71
2025-02-28,Fev,8161.38,7374.42,-786.96,-9.64
2025-03-31,Mar,8623.64,8144.82,-478.81,-5.55
2025-04-30,Abr,10024.71,7920.52,-2104.18,-20.99
2025-05-31,Mai,8566.48,8152.79,-413.70,-4.83
2025-06-30,Jun,6429.85,7893.55,1463.69,22.76
2025-07-31,Jul,5571.10,8188.91,2617.81,46.99
2025-08-31,Ago,6078.28,8136.80,2058.52,33.87
2025-09-30,Set,6840.02,7913.79,1073.77,15.70
2025-10-31,Out,8329.07,8174.98,-154.10,-1.85


# Conclusão

Para rodar outro cenário, ajuste os parâmetros da seção **2. Parâmetros** (ano, mês e edificação).

O restante do fluxo permanece o mesmo: carregar histórico, gerar previsão, salvar CSV e avaliar resultados.


# 8. Ajustes específicos para escola (monthly school)

Esta seção final aplica regras adicionais para o caso da escola, alinhadas ao entendimento do consumo desse prédio no primeiro notebook:
- uso de **feriados nacionais BR**,
- redução de peso para meses de férias (`jan` e `dez`),
- fator mensal com controle de meses de pico.

Ao final, a comparação anual mostra três séries:
- **Real**,
- **Previsão Inicial**,
- **Previsão Ajustada Escola**.


In [ ]:
# Ajustes do monthly school + gráfico comparativo (real x inicial x ajustado)

from datetime import date, timedelta

VACATION_MONTHS = {1, 12}
PEAK_TOLERANCE = 1.05  # Define que um mês é "pico" se for 5% maior que a média normal

def easter_sunday_nb(year: int) -> date:
    a = year % 19
    b = year // 100
    c = year % 100
    d = b // 4
    e = b % 4
    f = (b + 8) // 25
    g = (b - f + 1) // 3
    h = (19 * a + b - d - g + 15) % 30
    i = c // 4
    k = c % 4
    l = (32 + 2 * e + 2 * i - h - k) % 7
    m = (a + 11 * h + 22 * l) // 451
    month = (h + l - 7 * m + 114) // 31
    day = ((h + l - 7 * m + 114) % 31) + 1
    return date(year, month, day)


def brazil_holidays_nb(year: int) -> set[date]:
    easter = easter_sunday_nb(year)

    return {
        date(year, 1, 1),
        date(year, 4, 21),
        date(year, 5, 1),
        date(year, 9, 7),
        date(year, 10, 12),
        date(year, 11, 2),
        date(year, 11, 15),
        date(year, 11, 20),
        date(year, 12, 25),
        easter - timedelta(days=48),
        easter - timedelta(days=47),
        easter - timedelta(days=2),
        easter + timedelta(days=60),
    }



def holiday_index_for_dates_nb(idx: pd.DatetimeIndex) -> pd.DatetimeIndex:
    years = sorted(set(idx.year.tolist()))
    holidays = []
    for year in years:
        holidays.extend(pd.to_datetime(list(brazil_holidays_nb(int(year)))))
    return pd.DatetimeIndex(sorted(set(holidays))).normalize()



def mark_holidays_nb(idx: pd.DatetimeIndex) -> pd.Series:
    holiday_index = holiday_index_for_dates_nb(idx)
    return pd.Series(idx.normalize().isin(holiday_index).astype(int), index=idx)



def build_profile_school_adjusted(history: pd.DataFrame, target_col: str):
    hist = history.copy()
    hist["month"] = hist.index.month
    hist["weekday"] = hist.index.dayofweek
    hist["slot_15m"] = hist.index.hour * 4 + (hist.index.minute // 15)
    hist["is_holiday"] = mark_holidays_nb(hist.index)

    # --- Identificação Dinâmica de Férias e Picos ---
    month_means = hist.groupby("month")[target_col].mean()
    non_vac_mask = ~hist["month"].isin(VACATION_MONTHS)

    # Média letiva "normal" como base de comparação
    baseline_mean = float(hist.loc[non_vac_mask, target_col].mean()) if non_vac_mask.any() else float(month_means.mean())

    month_weights = {}
    month_factors = {}

    for month in range(1, 13):
        if month in month_means.index and baseline_mean > 0:
            m_mean = float(month_means.loc[month])
            ratio = m_mean / baseline_mean

            if month in VACATION_MONTHS:
                # Férias dinâmicas (peso reduzido, fator de redução)
                month_weights[month] = max(0.01, min(1.0, ratio))
                month_factors[month] = max(0.0, min(1.0, ratio))

            elif ratio >= PEAK_TOLERANCE:
                # Pico dinâmico detectado! (ex: calor extremo)
                # Peso inverso para não inflar a base, fator de aumento na projeção
                month_weights[month] = 1.0 / ratio
                month_factors[month] = ratio

            else:
                # Mês letivo normal
                month_weights[month] = 1.0
                month_factors[month] = 1.0

        else:
            month_weights[month] = 1.0
            month_factors[month] = 1.0

    # Aplica os pesos calculados dinamicamente
    hist["weight"] = hist["month"].map(month_weights).fillna(1.0).astype(float)

    def weighted_mean_by(group_cols: list[str]) -> pd.Series:
        grouped = hist.groupby(group_cols)
        weighted_sum = grouped.apply(lambda g: float((g[target_col] * g["weight"]).sum()))
        weight_sum = grouped["weight"].sum()
        return (weighted_sum / weight_sum).astype(float)

    profile_holiday_weekday_slot = weighted_mean_by(["is_holiday", "weekday", "slot_15m"])
    profile_weekday_slot_adj = weighted_mean_by(["weekday", "slot_15m"])
    profile_slot_adj = weighted_mean_by(["slot_15m"])
    global_mean_adj = float((hist[target_col] * hist["weight"]).sum() / hist["weight"].sum())

    return (
        profile_holiday_weekday_slot,
        profile_weekday_slot_adj,
        profile_slot_adj,
        global_mean_adj,
        month_factors,
    )

def forecast_month_school_adjusted(
    future_idx: pd.DatetimeIndex,
    profile_holiday_weekday_slot: pd.Series,
    profile_weekday_slot_adj: pd.Series,
    profile_slot_adj: pd.Series,
    global_mean_adj: float,
    month_factors: dict[int, float],
) -> pd.DataFrame:

    future = pd.DataFrame(index=future_idx)
    future["is_holiday"] = mark_holidays_nb(future.index)
    future["weekday"] = future.index.dayofweek
    future["slot_15m"] = future.index.hour * 4 + (future.index.minute // 15)

    def predict_row(row):
        holiday_key = (int(row["is_holiday"]), int(row["weekday"]), int(row["slot_15m"]))
        if holiday_key in profile_holiday_weekday_slot.index:
            return float(profile_holiday_weekday_slot.loc[holiday_key])

        key = (int(row["weekday"]), int(row["slot_15m"]))
        if key in profile_weekday_slot_adj.index:
            return float(profile_weekday_slot_adj.loc[key])
        
        slot = int(row["slot_15m"])
        if slot in profile_slot_adj.index:
            return float(profile_slot_adj.loc[slot])

        return global_mean_adj

    future["consumption_kwh_pred"] = future.apply(predict_row, axis=1)
    # Aplica os fatores mensais dinâmicos (reduz as férias, aumenta os picos)

    future["month_factor"] = future.index.month.map(month_factors).fillna(1.0).astype(float)
    future["consumption_kwh_pred"] = future["consumption_kwh_pred"] * future["month_factor"]

    return future[["consumption_kwh_pred"]]



# =========================================================
# Visão anual comparativa
# =========================================================

ano_analise = TARGET_YEAR
future_idx_annual = pd.date_range(
    start=pd.Timestamp(year=ano_analise, month=1, day=1),
    end=pd.Timestamp(year=ano_analise, month=12, day=31, hour=23, minute=45),
    freq="15min"
)

# Previsão inicial (já existente no notebook)
profile_weekday_slot_base, profile_slot_base, global_mean_base = build_profile(history, TARGET_COL)
forecast_annual_base = forecast_month(
    future_idx_annual, profile_weekday_slot_base, profile_slot_base, global_mean_base
)

forecast_monthly_base = forecast_annual_base["consumption_kwh_pred"].resample("ME").sum()

# Previsão ajustada para escola (agora com Férias e Picos Dinâmicos)
(
    profile_holiday_weekday_slot,
    profile_weekday_slot_adj,
    profile_slot_adj,
    global_mean_adj,
    month_factors
) = build_profile_school_adjusted(history, TARGET_COL)

forecast_annual_adj = forecast_month_school_adjusted(
    future_idx_annual,
    profile_holiday_weekday_slot,
    profile_weekday_slot_adj,
    profile_slot_adj,
    global_mean_adj,
    month_factors,
)

forecast_monthly_adj = forecast_annual_adj["consumption_kwh_pred"].resample("ME").sum()

# Preparação dos dados reais para o gráfico
actual_annual = history.loc[future_idx_annual.min():future_idx_annual.max(), [TARGET_COL]]
actual_monthly = actual_annual[TARGET_COL].resample("ME").sum() if not actual_annual.empty else pd.Series(dtype=float)

# Consolidação para o Plotly

df_plot_school = pd.DataFrame(index=forecast_monthly_base.index)
df_plot_school["Previsão Inicial"] = forecast_monthly_base.values
df_plot_school["Previsão Ajustada Escola"] = forecast_monthly_adj.reindex(df_plot_school.index).values
df_plot_school["Real"] = actual_monthly.reindex(df_plot_school.index)

MESES_PT = {1: "Jan", 2: "Fev", 3: "Mar", 4: "Abr", 5: "Mai", 6: "Jun", 7: "Jul", 8: "Ago", 9: "Set", 10: "Out", 11: "Nov", 12: "Dez"}

df_plot_school["Mes_Nome"] = df_plot_school.index.month.map(MESES_PT)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_plot_school["Mes_Nome"],
    y=df_plot_school["Real"],
    mode="lines+markers",
    name="Consumo Real",
    line=dict(color="#2563eb", width=3)
))

fig.add_trace(go.Scatter(
    x=df_plot_school["Mes_Nome"],
    y=df_plot_school["Previsão Inicial"],
    mode="lines+markers",
    name="Previsão Inicial",
    line=dict(color="#ef4444", width=3, dash="dot")
))

fig.add_trace(go.Scatter(
    x=df_plot_school["Mes_Nome"],
    y=df_plot_school["Previsão Ajustada Escola"],
    mode="lines+markers",
    name="Previsão Ajustada Escola",
    line=dict(color="#16a34a", width=3)
))



fig.update_layout(
    title=f"<b>Análise Anual ({ano_analise}): Real x Previsão Inicial x Previsão Ajustada</b><br><sup>Edificação: {BUILDING_FILTER}</sup>",
    xaxis_title="Mês do Ano",
    yaxis_title="Consumo Total Mensal (kWh)",
    plot_bgcolor="white",
    hovermode="x unified",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.update_yaxes(gridcolor="whitesmoke", tickformat=",")
fig.update_xaxes(showgrid=False)
fig.show()


/tmp/ipython-input-396/3701673567.py:100: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

/tmp/ipython-input-396/3701673567.py:100: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

/tmp/ipython-input-396/3701673567.py:100: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the oper

In [29]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

df_metrics = pd.DataFrame()
df_metrics["Mês"] = df_plot_school["Mes_Nome"]
df_metrics["Consumo Real (kWh)"] = df_plot_school["Real"]
df_metrics["Consumo Previsto (kWh)"] = df_plot_school["Previsão Ajustada Escola"]


df_metrics["Erro (kWh)"] = df_metrics["Consumo Previsto (kWh)"] - df_metrics["Consumo Real (kWh)"]
df_metrics["Erro Percentual (%)"] = np.where(
    df_metrics["Consumo Real (kWh)"] != 0,
    (df_metrics["Erro (kWh)"] / df_metrics["Consumo Real (kWh)"]) * 100,
    np.nan
)

# Arredondando os valores para facilitar a leitura
df_metrics = df_metrics.round({
    "Consumo Real (kWh)": 2,
    "Consumo Previsto (kWh)": 2,
    "Erro (kWh)": 2,
    "Erro Percentual (%)": 2
})

def format_br(val):
    if pd.isna(val):
        return "-"
    return f"{val:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

display(df_metrics)

,Mês,Consumo Real (kWh),Consumo Previsto (kWh),Erro (kWh),Erro Percentual (%)
2025-01-31,Jan,949.52,905.61,-43.91,-4.62
2025-02-28,Fev,1324.74,1214.53,-110.21,-8.32
2025-03-31,Mar,1604.51,1339.16,-265.35,-16.54
2025-04-30,Abr,1668.56,1391.66,-276.91,-16.60
2025-05-31,Mai,1711.78,1448.49,-263.28,-15.38
2025-06-30,Jun,1263.92,1263.99,0.08,0.01
2025-07-31,Jul,1458.96,1369.17,-89.78,-6.15
2025-08-31,Ago,1344.15,1312.64,-31.52,-2.34
2025-09-30,Set,1418.07,1317.99,-100.08,-7.06
2025-10-31,Out,1391.03,1365.13,-25.90,-1.86
